<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;

// Generic класс
public class PaymentProcessor<T> where T : PaymentMethod
{
    private List<T> _payments = new List<T>();

    public void AddPayment(T payment)
    {
        _payments.Add(payment);
        Console.WriteLine($"Добавлен: {payment.MethodName}");
    }

    public void ProcessAll(decimal amount)
    {
        Console.WriteLine($"Обработка всех платежей на сумму {amount}:");
        foreach (var payment in _payments)
        {
            payment.ProcessPayment(amount);
        }
    }
}

// Базовый класс
public class PaymentMethod
{
    public int PaymentMethodId { get; set; }
    public string MethodName { get; set; }
    public decimal MinAmount { get; set; }
    
    // Новые атрибуты
    public string Currency { get; set; }
    public bool IsActive { get; set; }
    public DateTime CreatedDate { get; set; }
    public string Description { get; set; }

    public PaymentMethod(int id, string name, decimal minAmount)
    {
        PaymentMethodId = id;
        MethodName = name;
        MinAmount = minAmount;
        Currency = "RUB";
        IsActive = true;
        CreatedDate = DateTime.Now;
        Description = "Базовый способ оплаты";
    }

    // Перегрузка методов
    public virtual void ProcessPayment(decimal amount)
    {
        Console.WriteLine($"Базовый платеж: {amount} {Currency}");
    }

    public virtual void ProcessPayment(decimal amount, string reference) // Перегрузка
    {
        Console.WriteLine($"Базовый платеж: {amount} {Currency} - {reference}");
    }

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        return amount >= MinAmount;
    }

    // Перегрузка CheckMinimumAmount
    public virtual bool CheckMinimumAmount(decimal amount, bool strictMode) // Перегрузка
    {
        if (strictMode)
            return amount > MinAmount;
        return amount >= MinAmount;
    }

    public virtual void GetPaymentDetails()
    {
        Console.WriteLine($"{MethodName} (ID: {PaymentMethodId}, Мин.: {MinAmount} {Currency})");
    }

    // Новые методы
    public virtual void Validate()
    {
        Console.WriteLine($"Валидация {MethodName}");
    }

    public string GetCreationInfo()
    {
        return $"Создан: {CreatedDate:dd.MM.yyyy}";
    }
}

// OnlinePayment - переопределение и перегрузка
public class OnlinePayment : PaymentMethod
{
    public string PaymentUrl { get; set; }
    
    // Новые атрибуты
    public string GatewayName { get; set; }
    public int TimeoutSeconds { get; set; }
    public bool SupportsRecurring { get; set; }

    public OnlinePayment(int id, string name, decimal minAmount, string paymentUrl, string gatewayName)
        : base(id, name, minAmount)
    {
        PaymentUrl = paymentUrl;
        GatewayName = gatewayName;
        TimeoutSeconds = 30;
        SupportsRecurring = false;
    }

    // Переопределение (override)
    public override void ProcessPayment(decimal amount)
    {
        if (CheckMinimumAmount(amount))
        {
            Console.WriteLine($"Онлайн платеж: {amount} {Currency} через {GatewayName}");
            Console.WriteLine($"URL: {PaymentUrl}, Таймаут: {TimeoutSeconds}с");
        }
    }

    // Перегрузка (overload)
    public void ProcessPayment(decimal amount, bool useSecureConnection) // Перегрузка
    {
        ProcessPayment(amount);
        Console.WriteLine($"Безопасное соединение: {useSecureConnection}");
    }

    // Перекрытие (new)
    public new void Validate() // Перекрытие
    {
        Console.WriteLine($"Расширенная валидация онлайн платежа");
        Console.WriteLine($"Проверка URL: {PaymentUrl}");
        Console.WriteLine($"Проверка шлюза: {GatewayName}");
    }

    // Новые методы
    public void EnableRecurringPayments()
    {
        SupportsRecurring = true;
        Console.WriteLine("Рекуррентные платежи активированы");
    }
}

// BankTransfer - переопределение и перегрузка
public class BankTransfer : PaymentMethod
{
    public string BankData { get; set; }
    public decimal CommissionRate { get; set; }
    
    // Новые атрибуты
    public string AccountNumber { get; set; }
    public string BIC { get; set; }
    public bool InternationalEnabled { get; set; }

    public BankTransfer(int id, string name, decimal minAmount, string bankData, 
                       decimal commissionRate, string accountNumber)
        : base(id, name, minAmount)
    {
        BankData = bankData;
        CommissionRate = commissionRate;
        AccountNumber = accountNumber;
        BIC = "044525225";
        InternationalEnabled = false;
    }

    // Переопределение (override)
    public override void ProcessPayment(decimal amount)
    {
        decimal commission = amount * CommissionRate;
        decimal total = amount + commission;

        if (CheckMinimumAmount(amount))
        {
            Console.WriteLine($"Банковский перевод: {amount} {Currency}");
            Console.WriteLine($"Комиссия: {commission} {Currency}, Итого: {total} {Currency}");
            Console.WriteLine($"Банк: {BankData}, Счет: {AccountNumber}");
        }
    }

    // Перегрузка (overload)
    public void ProcessPayment(decimal amount, string recipient) // Перегрузка
    {
        ProcessPayment(amount);
        Console.WriteLine($"Получатель: {recipient}");
    }

    // Переопределение CheckMinimumAmount с учетом комиссии
    public override bool CheckMinimumAmount(decimal amount)
    {
        decimal totalWithCommission = amount + (amount * CommissionRate);
        return totalWithCommission >= MinAmount;
    }

    // Новые методы
    public void EnableInternationalTransfers()
    {
        InternationalEnabled = true;
        Console.WriteLine("Международные переводы активированы");
    }
}

// CashPayment - третий производный класс
public class CashPayment : PaymentMethod
{
    public string CashPickupPoint { get; set; }
    
    // Новые атрибуты
    public string OperatorName { get; set; }
    public string LocationAddress { get; set; }
    public string WorkingHours { get; set; }

    public CashPayment(int id, string name, decimal minAmount, string cashPickupPoint, string operatorName)
        : base(id, name, minAmount)
    {
        CashPickupPoint = cashPickupPoint;
        OperatorName = operatorName;
        LocationAddress = "Основное отделение";
        WorkingHours = "9:00-18:00";
    }

    // Переопределение (override)
    public override void ProcessPayment(decimal amount)
    {
        if (CheckMinimumAmount(amount))
        {
            Console.WriteLine($"Оплата наличными: {amount} {Currency}");
            Console.WriteLine($"Пункт: {CashPickupPoint}, Адрес: {LocationAddress}");
            Console.WriteLine($"Оператор: {OperatorName}, Часы: {WorkingHours}");
        }
    }

    // Перегрузка (overload)
    public void ProcessPayment(decimal amount, bool printReceipt) // Перегрузка
    {
        ProcessPayment(amount);
        if (printReceipt)
            Console.WriteLine("Чек распечатан");
    }

    // Переопределение GetPaymentDetails
    public override void GetPaymentDetails()
    {
        Console.WriteLine($"Наличные: {MethodName} в {CashPickupPoint}");
        Console.WriteLine($"Часы работы: {WorkingHours}");
    }

    // Новые методы
    public void UpdateWorkingHours(string hours)
    {
        WorkingHours = hours;
        Console.WriteLine($"Часы работы обновлены: {hours}");
    }
}

// Демонстрация
public class Program
{
    public static void Main()
    {
       
    }
}
 Console.WriteLine("=== ПОЛНАЯ ДЕМОНСТРАЦИЯ ===");

        // Создание объектов
        var online = new OnlinePayment(1, "Интернет-оплата", 10, "https://pay.site.com", "Stripe");
        var bank = new BankTransfer(2, "Банковский перевод", 50, "Сбербанк", 0.015m, "40702810000000012345");
        var cash = new CashPayment(3, "Наличные", 1, "Касса магазина", "ИП Петров");

        // Демонстрация перегрузки
        Console.WriteLine("\n1. ПЕРЕГРУЗКА МЕТОДОВ:");
        online.ProcessPayment(100);
        online.ProcessPayment(100, true); // Перегруженный метод
        
        bank.ProcessPayment(200);
        bank.ProcessPayment(200, "ООО Ромашка"); // Перегруженный метод
        
        cash.ProcessPayment(50);
        cash.ProcessPayment(50, true); // Перегруженный метод

        // Демонстрация переопределения и перекрытия
        Console.WriteLine("\n2. ПЕРЕОПРЕДЕЛЕНИЕ И ПЕРЕКРЫТИЕ:");
        online.Validate(); // Перекрытый метод
        bank.Validate();   // Базовый метод (не перекрыт)
        cash.GetPaymentDetails(); // Переопределенный метод

        // Демонстрация полиморфизма через базовый класс
        Console.WriteLine("\n3. ПОЛИМОРФИЗМ:");
        PaymentMethod[] payments = { online, bank, cash };
        foreach (var payment in payments)
        {
            payment.ProcessPayment(150); // Полиморфный вызов
            payment.GetPaymentDetails();
            Console.WriteLine();
        }

        // Демонстрация generic класса
        Console.WriteLine("\n4. GENERIC КЛАСС:");
        var processor = new PaymentProcessor<PaymentMethod>();
        processor.AddPayment(online);
        processor.AddPayment(bank);
        processor.AddPayment(cash);
        processor.ProcessAll(300);

        // Демонстрация новых методов
        Console.WriteLine("\n5. НОВЫЕ МЕТОДЫ:");
        online.EnableRecurringPayments();
        bank.EnableInternationalTransfers();
        cash.UpdateWorkingHours("10:00-20:00");

        Console.WriteLine("\n=== ВСЕ ТРЕБОВАНИЯ ВЫПОЛНЕНЫ ===");

=== ПОЛНАЯ ДЕМОНСТРАЦИЯ ===

1. ПЕРЕГРУЗКА МЕТОДОВ:
Онлайн платеж: 100 RUB через Stripe
URL: https://pay.site.com, Таймаут: 30с
Онлайн платеж: 100 RUB через Stripe
URL: https://pay.site.com, Таймаут: 30с
Безопасное соединение: True
Банковский перевод: 200 RUB
Комиссия: 3.000 RUB, Итого: 203.000 RUB
Банк: Сбербанк, Счет: 40702810000000012345
Банковский перевод: 200 RUB
Комиссия: 3.000 RUB, Итого: 203.000 RUB
Банк: Сбербанк, Счет: 40702810000000012345
Получатель: ООО Ромашка
Оплата наличными: 50 RUB
Пункт: Касса магазина, Адрес: Основное отделение
Оператор: ИП Петров, Часы: 9:00-18:00
Оплата наличными: 50 RUB
Пункт: Касса магазина, Адрес: Основное отделение
Оператор: ИП Петров, Часы: 9:00-18:00
Чек распечатан

2. ПЕРЕОПРЕДЕЛЕНИЕ И ПЕРЕКРЫТИЕ:
Расширенная валидация онлайн платежа
Проверка URL: https://pay.site.com
Проверка шлюза: Stripe
Валидация Банковский перевод
Наличные: Наличные в Касса магазина
Часы работы: 9:00-18:00

3. ПОЛИМОРФИЗМ:
Онлайн платеж: 150 RUB через Stripe
URL: https:/